In [9]:
import astropy.cosmology.units as u
from astropy.cosmology import Planck15 as cosmo
import numpy as np
from astropy.io import fits
import pandas as pd

In [7]:
data = fits.open('Rommulus_withLight_Mock_i2_SNIa_z1_cusp.fits')
data.info()

Filename: Rommulus_withLight_Mock_i2_SNIa_z1_cusp.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      37   (1039, 1039, 6)   float64   


In [8]:
data[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                 1039                                                  
NAXIS2  =                 1039                                                  
NAXIS3  =                    6                                                  
EXTEND  =                    T                                                  
BUNIT   = 'electron/s'                                                          
TELESCOP= 'HST     '                                                            
INSTRUME= 'WFC3    '                                                            
FILTER  = 'SDSS-r (F625W)'                                                      
PHOTFLAM=              1.2E-19 / erg/cm**2/AA/electron                          
WL_PIVOT=               6266

In [2]:
def flux2mag(M, z, flux):
    PHOTFLAM = data[0].header['PHOTFLAM']
    WL_PIVOT=  data[0].header['WL_PIVOT']
    
    #find the zero point
    f_PHOTZPT = 2.49767E-9 #erg/cm**2/s/AA, filter: SDSS-r
    f0_PHOTZPT = f_PHOTZPT * (1E-3) * WL_PIVOT
    PHOTZPT = (2.5)*np.log10(f0_PHOTZPT)
    
    #theoratical apparent magnitude of this SNIa
    Dls = (cosmo.luminosity_distance(z)).value*np.power(10,6) #luminosity distance of SNIa
    ms = M + 5*np.log10(Dls/10)
    print('ms = ' + str(ms))
    
    #flux to apparent magnitude
    flux0 = flux*PHOTFLAM*WL_PIVOT #erg/cm**2/s
    f = flux0*(1E-3) #joules/m**2/s
    m = -(2.5)*np.log10(f) + PHOTZPT
    print('m = ' + str(m))
    
    #convert magtitude difference to magnification
    m_diff = m-ms
    base = np.power(10, 1/2.5)
    mag = np.power(base,abs(m_diff))
    
    return mag

In [10]:
pred = pd.read_csv('pred.csv')
pred

,x,x_err,y,y_err,flux,flux_err,image
0,61.403103,0.018381,49.589840,0.018376,11.870106,0.183270,1
1,73.750355,0.016142,66.015834,0.016327,16.100805,0.216334,2
2,51.829783,0.019070,68.241797,0.018977,14.733413,0.232599,3
3,62.520786,0.008990,73.452096,0.008993,37.469106,0.283404,4


In [14]:
######change here########
M = data[0].header['HIERARCH Source Absolute AB Magnitude']
z = data[0].header['Z_SOURCE']
flux = np.array(pred['flux'])

In [15]:
mag = np.round(flux2mag(M, z, flux),3)
print('The magnifications are '+ str(mag) + '.')

ms = 24.859927839860408
m = [23.10974805 22.77876558 22.8751261  21.86170121]
The magnifications are [ 5.013  6.799  6.222 15.823].


In [16]:
max_flux = np.max(flux)
flux_ratio = flux/max_flux
print('The flux ratios are '+ str(np.round(flux_ratio,3)) + '.')

max_mag = np.max(mag)
mag_ratio = mag/max_mag
print('The magnification ratios are '+ str(np.round(mag_ratio,3)) + '.')

The flux ratios are [0.317 0.43  0.393 1.   ].
The magnification ratios are [0.317 0.43  0.393 1.   ].
